<a href="https://colab.research.google.com/github/acl-jianghao/Langchain-demos/blob/main/07_sequentialChain_tools_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [141]:
!pip install -Uqq openai langchain tiktoken colab-env pymupdf faiss-cpu google-search-results
# !pip install -Uqq pyspark-ai pyspark
!pip install -Uqq chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.0/399.0 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━

In [37]:
import os
import colab_env

In [ ]:
# from langchain.agents import load_tools

# tools = load_tools(["serpapi"])
# tools.append()

In [10]:
# from langchain.utilities import SerpAPIWrapper

# search = SerpAPIWrapper()

# search.run("what is the weather of Hangzhou of next 7 days?")

'Precipitation: Rain: 0.01 Snow: 0. Precipitation Chance: 11%. E. Wind: 7 mph. Jul 24 – Jul 30Jul 31 – Aug 6Aug 7 – Aug 13. See weather overview ...'

In [38]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')
serpapi_api_key = os.getenv('SERPAPI_API_KEY')

In [107]:
# from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.agents import load_tools, initialize_agent


# llm=ChatOpenAI(temperature=0, verbose=True)
llm=OpenAI(temperature=0, verbose=True)
# tools = load_tools(["serpapi"], llm, serpapi_api_key=serpapi_api_key)
# agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# result = agent.run("please list the weather of each day in Hangzhou of the next 7 days?")

In [108]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain


# Prompt templates
serp_template = PromptTemplate(
    input_variables = ['city'],
    template="""Please list the weather, temperature and air quality of each day in {city} of
            the next 7 days and response should be a string in code block"""
)

tell_template = PromptTemplate(
    input_variables = ['string'],
    template="""Tell me the average temperature in {string},
            the answer should only have the average temperature in Chinese """
)



In [109]:
weather_chain = LLMChain(llm=llm, prompt=serp_template, verbose=False, output_key='string')
tell_chain = LLMChain(llm=llm, prompt=tell_template, verbose=False, output_key='average_temperature')


sequential_chain = SequentialChain(
    chains = [weather_chain, tell_chain],
    input_variables=['city'],
    output_variables=['string', 'average_temperature'],
    verbose=True)

# sequential_chain = SequentialChain(
#     chains = [weather_chain],
#     input_variables=['city'],
#     output_variables=['json'],
#     verbose=True)

In [110]:
sequential_chain('Hangzhou')



> Entering new SequentialChain chain...

> Finished chain.


{'city': 'Hangzhou',
 'string': '\n\nDay 1: Weather: Sunny, Temperature: 25°C, Air Quality: Good\nDay 2: Weather: Cloudy, Temperature: 22°C, Air Quality: Moderate\nDay 3: Weather: Rainy, Temperature: 20°C, Air Quality: Poor\nDay 4: Weather: Partly Cloudy, Temperature: 23°C, Air Quality: Good\nDay 5: Weather: Sunny, Temperature: 25°C, Air Quality: Moderate\nDay 6: Weather: Cloudy, Temperature: 22°C, Air Quality: Poor\nDay 7: Weather: Partly Cloudy, Temperature: 24°C, Air Quality: Good',
 'average_temperature': '\n\n平均温度：22.7°C'}

In [57]:
# from pyspark_ai import SparkAI

# spark_ai = SparkAI(llm=llm, verbose=True)

# spark_ai.activate()


tools + agent

In [150]:
from langchain.agents import load_tools, initialize_agent

tools = load_tools(["serpapi"], llm, serpapi_api_key=serpapi_api_key)


# agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# result = agent.run("please list the weather of each day in Hangzhou of the next 7 days?")

In [151]:
from langchain.tools import BaseTool

class CalculateAverage(BaseTool):
    name = "Calculate Average"
    description = 'use this tool to calculate the average value.'

    def _run(self, expr: str):
        return 10.1

    def _arun(self, query: str):
        raise NotImplementedError("Async function is not ready yet")

tools.append(CalculateAverage())


tools

[Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='3d13bfdc6f306fb453379219014847c07d5d945f49e77eef4d819b5421d57c2b', aiosession=None)>, coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='3d13bfdc6f306fb453379219014847c07d5d945f49e77eef4d819b5421d57c2b', aiosession=None)>),
 CalculateAverage(name='Calculate Average', description='use this tool to calculate the ave

In [ ]:
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=True
)

In [132]:
agent.run("please tell me the temprature average of Hangzhou in the next 7 days")



> Entering new AgentExecutor chain...
 I need to find a reliable source of weather information
Action: Search
Action Input: "Hangzhou weather forecast 7 days"
Observation: Precipitation: Rain: 0.01 Snow: 0. Precipitation Chance: 11%. E. Wind: 7 mph. Jul 24 – Jul 30Jul 31 – Aug 6Aug 7 – Aug 13. See weather overview ...
Thought: I need to look for the temperature averages
Action: Search
Action Input: "Hangzhou temperature forecast 7 days"
Observation: 2 Week Extended Forecast in Hangzhou, Zhejiang, China ; 85 / 78 °F · 85 / 78 °F · 87 / 79 °F ...
Thought: I now know the final answer
Final Answer: The average temperature in Hangzhou over the next 7 days is approximately 85°F (29°C).

> Finished chain.


'The average temperature in Hangzhou over the next 7 days is approximately 85°F (29°C).'

In [144]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma


# index_root_dir = '/content/gdrive/MyDrive/index/'

embedding = OpenAIEmbeddings()
chroma_index = Chroma(
    persist_directory='./.chroma_index', embedding_function=embedding)

In [152]:
from langchain.experimental import AutoGPT
from langchain.chat_models import ChatOpenAI


agent = AutoGPT.from_llm_and_tools(
    ai_name="link",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0),
    memory=chroma_index.as_retriever()
)
# Set verbose to be true
agent.chain.verbose = True

tools

[Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='3d13bfdc6f306fb453379219014847c07d5d945f49e77eef4d819b5421d57c2b', aiosession=None)>, coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='3d13bfdc6f306fb453379219014847c07d5d945f49e77eef4d819b5421d57c2b', aiosession=None)>),
 CalculateAverage(name='Calculate Average', description='use this tool to calculate the ave

In [153]:
agent.run(["Calculate the average temprature of Hangzhou next 7 days"])



> Entering new LLMChain chain...
Prompt after formatting:
System: You are link, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Calculate the average temprature of Hangzhou next 7 days


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query., args json schema: {"tool_input": {"type": "string"}}
2. Calculate Aver


> Finished chain.
{
    "thoughts": {
        "text": "Since I have gathered the information about the average temperature in Hangzhou for the next 7 days, I can now use the 'Calculate Average' command to calculate the average temperature.",
        "reasoning": "I have the necessary data to calculate the average temperature in Hangzhou for the next 7 days.",
        "plan": "- Use the 'Calculate Average' command with the expression to calculate the average temperature.",
        "criticism": "N/A",
        "speak": "I will calculate the average temperature in Hangzhou for the next 7 days."
    },
    "command": {
        "name": "Calculate Average",
        "args": {
            "expr": "93 + 92 + 89 + 88 + 87 + 86 + 85 / 7"
        }
    }
}


> Entering new LLMChain chain...


Prompt after formatting:
System: You are link, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Calculate the average temprature of Hangzhou next 7 days


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query., args json schema: {"tool_input": {"type": "string"}}
2. Calculate Average: use this tool to calculate the



> Entering new LLMChain chain...
Prompt after formatting:
System: You are link, Assistant
Your decisions must always be made independently without seeking user assistance.
Play to your strengths as an LLM and pursue simple strategies with no legal complications.
If you have completed all your tasks, make sure to use the "finish" command.

GOALS:

1. Calculate the average temprature of Hangzhou next 7 days


Constraints:
1. ~4000 word limit for short term memory. Your short term memory is short, so immediately save important information to files.
2. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
3. No user assistance
4. Exclusively use the commands listed in double quotes e.g. "command name"

Commands:
1. Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query., args json schema: {"tool_input": {"type": "string"}}
2. Calculate Aver

'I have finished calculating the average temperature in Hangzhou for the next 7 days.'